In [14]:
import numpy as np
import pandas as pd
from scipy.stats import bernoulli
import itertools
import networks

In [58]:
z0 = [3, 1, 2, 1, 3, 1, 2, 2, 2, 3, 3, 2, 1, 1, 3]
p = 0.6
q = 0.1


def sample_maker(p, q, z0):
    A = [[0] * 15 for i in range(15)]
    for i in range(0, 15):
        A[i][i] = 1
        for j in range(0, i):
            if z0[i] == z0[j]:
                A[i][j] = np.random.binomial(1, p, size=1)[0]
                #bernoulli.rvs(p, size=1)
            else:
                A[i][j] = np.random.binomial(1, q, size=1)[0]
            A[j][i] = A[i][j]
    return A


A = [0] * 10
for i in range(10):
    A[i] = sample_maker(p, 1, z0)
    print(A[i])


[[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1], [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1], [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1], [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1]]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1], [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1], [1, 1, 1, 1

In [2]:
a = [1, 1, 1, 2, 3, 3, 2, 3, 2]
b = [1, 1, 3, 2, 3, 1, 2, 3, 2]
x = len(a)
x

9

In [3]:
def hamming(z1, z2):
    ans = 0
    n = len(z1)
    for i in range(0, n):
        if z1[i] != z2[i]:
            ans += 1
    return ans


hamming(a, b)

2

In [4]:
def min_hamming(z1, z2):
    z1_values = list(pd.Series(z1).value_counts().index)
    permutations = list(itertools.permutations(z1_values))
    min = 1000000000
    for i in range(0, len(permutations)):
        tmp = [0] * len(z2)
        for j in range(0, len(z2)):
            tmp[j] = permutations[i][z2[j] - 1]
        print(tmp)
        if min > hamming(tmp, z1):
            min = hamming(tmp, z1)
    return min


min_hamming(a, b)

[1, 1, 3, 2, 3, 1, 2, 3, 2]
[1, 1, 2, 3, 2, 1, 3, 2, 3]
[2, 2, 3, 1, 3, 2, 1, 3, 1]
[2, 2, 1, 3, 1, 2, 3, 1, 3]
[3, 3, 2, 1, 2, 3, 1, 2, 1]
[3, 3, 1, 2, 1, 3, 2, 1, 2]


2

In [12]:
x = np.array([0] * 9).reshape(3, 3)
x = [[0] * 2 for row in range(2)]
x

[[0, 0], [0, 0]]